# 因果语言模型训练实例

## Step1 导入相关包

In [1]:
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
    BloomForCausalLM,
)

/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("./wiki_cn_filtered/")

In [3]:
ds

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [4]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## Step3 数据集处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-389m-zh")


def process_func(examples):
    contents = [e + tokenizer.eos_token for e in examples["completion"]]
    return tokenizer(
        contents, max_length=384, truncation=True
    )  # labels在DataCollatorForLanguageModeling

In [6]:
tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 10000
})

In [7]:
from torch.utils.data import DataLoader

dl = DataLoader(
    tokenized_ds,
    batch_size=2,
    collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [8]:
next(enumerate(dl))
# 3是pad
# 2是eos

(0,
 {'input_ids': tensor([[    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3

In [9]:
tokenizer.pad_token, tokenizer.pad_token_id

('<pad>', 3)

In [10]:
tokenizer.eos_token, tokenizer.eos_token_id

('</s>', 2)

## Step4 创建模型

In [11]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-389m-zh")

In [12]:
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(42437, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (l

In [13]:
from torch.nn import ModuleList

model.transformer.h = ModuleList([model.transformer.h[i] for i in range(0, 24, 6)])
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(42437, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-3): 4 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_

In [14]:
for param in model.transformer.word_embeddings.parameters():
    param.requires_grad = False

## Step5 配置训练参数

In [15]:
args = TrainingArguments(
    output_dir="./causal_lm",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    logging_steps=10,
    num_train_epochs=1,
    fp16=True,
)

## Step6 创建训练器

In [16]:
trainer = Trainer(
    args=args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

## Step7 模型训练

In [17]:
trainer.train()

  0%|          | 0/1250 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
  1%|          | 11/1250 [00:02<04:11,  4.93it/s]

{'loss': 25.0814, 'grad_norm': 27.7113037109375, 'learning_rate': 4.972e-05, 'epoch': 0.01}


  2%|▏         | 21/1250 [00:04<04:01,  5.09it/s]

{'loss': 9.8204, 'grad_norm': 14.930438995361328, 'learning_rate': 4.932e-05, 'epoch': 0.02}


  2%|▏         | 31/1250 [00:06<04:05,  4.96it/s]

...

100%|██████████| 1250/1250 [04:06<00:00,  5.26it/s]

{'loss': 5.679, 'grad_norm': 6.600950717926025, 'learning_rate': 1.2e-07, 'epoch': 1.0}


100%|██████████| 1250/1250 [04:07<00:00,  5.05it/s]

{'train_runtime': 247.5437, 'train_samples_per_second': 40.397, 'train_steps_per_second': 5.05, 'train_loss': 6.5083125213623045, 'epoch': 1.0}


TrainOutput(global_step=1250, training_loss=6.5083125213623045, metrics={'train_runtime': 247.5437, 'train_samples_per_second': 40.397, 'train_steps_per_second': 5.05, 'total_flos': 853652006830080.0, 'train_loss': 6.5083125213623045, 'epoch': 1.0})

## Step8 模型推理

In [18]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [28]:
model.config.n_layer = len(range(0, 24, 6))

In [31]:
pipe(
    "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安",
    max_length=128,
    do_sample=True,
    use_cache=False,  # 没有这个会报错
)

/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[{'generated_text': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安市西安市的一座地下文物馆，建有长安图书馆馆馆，位于西安大图书馆。\n历史\n西安东体育场\n西安地铁与西安地铁1号为西安地铁车站，位于西安地铁2号线3号线上。车站南侧，东地铁9号线2号线站台与月台，东侧，地下两层地面为西安西北路地下；地下2层2层9层，车站有100层2层9层层，高9层的12层，2层4层楼1层。\n历史\n1座车站3层7层"}]

In [33]:
pipe(
    "下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常",
    max_length=128,
    do_sample=True,
    use_cache=False,  # 没有这个会报错
    top_k=5,
)

/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[{'generated_text': '下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常丰富，在游戏中使用，游戏中的玩家，玩家将玩家玩家游戏，玩家将游戏游戏游戏，游戏玩家游戏，游戏玩家可以设定设定设定。游戏玩家可以游戏玩法玩法。\n玩家将玩家游戏游戏游戏，游戏玩法为游戏，游戏的游戏游戏游戏，游戏是游戏游戏，游戏游戏玩法。\n游戏游戏玩家玩家玩家游戏\n游戏玩法\n游戏\n游戏游戏是玩家游戏玩家游戏游戏，游戏玩家可以设定为玩家游戏游戏，游戏玩家玩家玩家玩家在游戏玩法玩法玩法，玩家玩家玩家玩家可以玩家'}]